In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import toolkitETL
import importlib
importlib.reload(toolkitETL)


<module 'toolkitETL' from 'c:\\Users\\pedro\\OneDrive\\Área de Trabalho\\Pedro\\Portifolio\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_base = "D:\\Mestrado_pedro"
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_biomap = os.path.join(diretorio_bronze, 'biomap')
diretorio_biomap_tsv = os.path.join(diretorio_biomap, 'tsv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_biomap_silver = os.path.join(diretorio_silver, 'biomap')
diretorio_biomap_tsv_silver = os.path.join(diretorio_biomap_silver, 'tsv')


## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)

# Bronze
os.makedirs(os.path.join(diretorio_bronze), exist_ok=True)
os.makedirs(os.path.join(diretorio_biomap), exist_ok=True)
os.makedirs(os.path.join(diretorio_biomap_tsv), exist_ok=True)

# Silver
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_biomap_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_biomap_tsv_silver), exist_ok=True)



In [3]:
arquivos_tsv = [arquivo for arquivo in os.listdir(diretorio_biomap_tsv) if arquivo.endswith("tsv")]
arquivos_tsv

['biomaps.tsv']

In [19]:
colunas_importantes = ['geocode', 'biome','level_4'] + [f'{num}' for num in range(2010,2022)]

column_map = {
    'geocode': 'id_municip',
    'Aquaculture': 'aquaculture', 
    'Beach and Dune': 'beach_and_dune', 
    'Citrus': 'citrus', 
    'Coffe': 'coffe', 
    'Cotton': 'cotton', 
    'Flooded Forest': 'flooded_forest', 
    'Forest Formation': 'forest_formation', 
    'Forest Plantation': 'forest_plantation', 
    'Grassland': 'grassland', 
    'Magrove': 'magrove', 
    'Mining': 'mining', 
    'Mosaic of Agriculture and Pasture': 'mosaic_of_agriculture_and_pasture', 
    'Non Observed': 'non_observed', 
    'Other Non Forest Natural Formation': 'other_non_forest_natural_formation', 
    'Other Non Vegetated Area': 'other_non_vegetated_area', 
    'Other Perennial Crops': 'other_perennial_crops', 
    'Other Temporary Crops': 'other_temporary_crops', 
    'Palm Oil': 'palm_oil', 
    'Pasture': 'pasture', 
    'Restinga Herbácea/Arbustiva': 'restinga_herbácea_arbustiva', 
    'Rice': 'rice', 
    'River, Lake and Ocean': 'river_lake_and_ocean', 
    'Rocky outcrop': 'rocky_outcrop', 
    'Salt flat': 'salt_flat', 
    'Savanna Formation': 'savanna_formation', 
    'Soy Beans': 'soy_beans', 
    'Sugar Cane': 'sugar_cane', 
    'Urban Infrastructure': 'urban_infrastructure', 
    'Wetland': 'wetland', 
    'Wooded Restinga': 'wooded_restinga'
}

In [27]:
lista_dfs = []


for arquivo_tsv in arquivos_tsv:
    diretorio_arquivo_tsv_bronze = os.path.join(diretorio_biomap_tsv, arquivo_tsv)
    df = pd.read_csv(diretorio_arquivo_tsv_bronze, sep='\t')
    df = df[colunas_importantes]

    df['geocode'] = df['geocode'].apply(lambda x: str(x)[:-1] if len(str(x)) == 7 else x)
    # Derretendo o DataFrame para que os anos virem uma única coluna
    df_melted = pd.melt(df, id_vars=['geocode', 'biome', 'level_4'], var_name='nu_ano', value_name='value')
    # Pivotando o DataFrame para que 'level_4' vire colunas
    df_pivoted = df_melted.pivot_table(index=['geocode', 'biome', 'nu_ano'], columns='level_4', values='value').reset_index()

    df_pivoted.columns.name = None
    df_pivoted = df_pivoted.rename(columns=column_map)

    lista_dfs.append(df_pivoted)



df_final = pd.concat(lista_dfs, ignore_index=True)

df_final["biomaps_id"] = df_final.index + 1
df_final.to_csv(os.path.join(diretorio_biomap_tsv_silver, 'biomap_tratado.tsv'), sep='\t', index=False)


In [4]:
# df_final = pd.read_csv(os.path.join(diretorio_biomap_tsv_silver, 'biomap_tratado.tsv'), sep='\t')



In [8]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)

caminho_arquivo = os.path.join(diretorio_biomap_tsv_silver, 'biomap_tratado.tsv')
toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'biomaps', delimiter='\t',
                                    diretorio_dotenv=diretorio_dotenv)
